In [1]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [6]:
a = torch.randn(3, 3)
print(a)
print(a.T)

tensor([[-0.4387, -1.2917,  0.2061],
        [ 0.4424, -0.6981,  0.0737],
        [ 0.4169,  0.1429,  0.2033]])
tensor([[-0.4387,  0.4424,  0.4169],
        [-1.2917, -0.6981,  0.1429],
        [ 0.2061,  0.0737,  0.2033]])


In [7]:
def yield_something(lst):
    for x in lst:
        yield x

generator = yield_something([1, 2, 3])

In [11]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [12]:
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLu(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        predictions = self.linear_relu_stack(x)
        return predictions
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()

# For the optimizer, we'll use SGD with a learning rate of 1e-3: it will adjust the parameters of the model using the gradients computed during backpropagation.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# SGD: Stochastic Gradient Descent



In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [67]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.199998  [    0/60000]
loss: 2.208095  [ 6400/60000]
loss: 2.184839  [12800/60000]
loss: 2.204555  [19200/60000]
loss: 2.164651  [25600/60000]
loss: 2.178029  [32000/60000]
loss: 2.167308  [38400/60000]
loss: 2.165127  [44800/60000]
loss: 2.153713  [51200/60000]
loss: 2.141772  [57600/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 2.144247 

Epoch 2
-------------------------------
loss: 2.147964  [    0/60000]
loss: 2.156060  [ 6400/60000]
loss: 2.123590  [12800/60000]
loss: 2.149909  [19200/60000]
loss: 2.091485  [25600/60000]
loss: 2.109032  [32000/60000]
loss: 2.093867  [38400/60000]
loss: 2.091308  [44800/60000]
loss: 2.074063  [51200/60000]
loss: 2.054495  [57600/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 2.061812 

Epoch 3
-------------------------------
loss: 2.068889  [    0/60000]
loss: 2.078368  [ 6400/60000]
loss: 2.034320  [12800/60000]
loss: 2.069765  [19200/60000]
loss: 1.986987  [25600/60000]
loss: 2.011277  [32000/600

In [23]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [24]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

/tmp/ipykernel_183724/2085806346.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [31]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
